In [6]:
import os
import torch
import re
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
from dotenv import load_dotenv

In [7]:
load_dotenv()
hf_token = os.getenv("HF_TOKEN")
if hf_token:
    login(token=hf_token)
else:
    raise ValueError("❌ Hugging Face Token not found. Please set HF_TOKEN in .env.")

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [8]:
def load_model(model_name="meta-llama/Llama-3.1-8B-Instruct"):
    device = "mps" if torch.backends.mps.is_available() else ("cuda" if torch.cuda.is_available() else "cpu")
    print(f"✅ Using device: {device}")

    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
    model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=True).to(device)
    print("🚀 Model and tokenizer loaded successfully!")

    return model, tokenizer

model, tokenizer = load_model()

✅ Using device: mps


/Users/zhengqiyao/.pyenv/versions/3.12.0/lib/python3.12/site-packages/transformers/models/auto/tokenization_auto.py:833: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/Users/zhengqiyao/.pyenv/versions/3.12.0/lib/python3.12/site-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
def generate_feedback(proposal, group):
    if group == "Improvement Feedback":
        prompt = f"""
You are an expert evaluator for hackathon proposals. Your task is to provide feedback based on the proposal.

--- New Proposal to Evaluate ---
Proposal: {proposal}

Please provide 3-4 concrete suggestions for improving the following solution. Each suggestion should be presented as a separate paragraph. Focus on the following aspects:

1. Identify existing weaknesses and explain why improvements are needed.
2. Propose clear, actionable steps to enhance effectiveness and innovation.
"""
    else:
        prompt = f"""
You are an expert evaluator for hackathon proposals. Your task is to provide feedback based on the proposal.

--- New Proposal to Evaluate ---
Proposal: {proposal}

Please identify 3-4 aspects of the following solution that are either redundant, highly similar to existing solutions, or uniquely innovative. Present the output as follows:

1. **1-2 aspects that are not novel:** Highlight features that resemble existing solutions and explain why they lack uniqueness.
2. **1-2 aspects that are novel and uncommon:** Identify distinctive features worth retaining and elaborate on their value.
"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(inputs["input_ids"], max_length=512, temperature=0.7)
    feedback = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return feedback

In [14]:
def clean_text(text):
    text = re.sub(r'[^\w\s.]', '', text)
    return text.strip()

In [15]:
proposal = input("Write your proposal here: ")
proposal = clean_text(proposal)

KeyboardInterrupt: Interrupted by user

In [ ]:
print("\n🟢 Improvement Feedback:")
improvement_feedback = generate_feedback(proposal, "Improvement Feedback")
print(improvement_feedback)

In [ ]:
print("\n🔵 Redundancy Feedback:")
redundancy_feedback = generate_feedback(proposal, "Redundancy Feedback")
print(redundancy_feedback)